In [2]:
# instaling the necessary drive
!python -m pip install pandas 

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------------ --------------------------- 3.7/11.5 MB 18.1 MB/s eta 0:00:01
   -------------------------- ------------- 7.6/11.5 MB 18.8 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 19.0 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 16.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------------- -------------------------- 4.2/12.6 MB 19.4 MB/s eta 0:00:01
   ------------------------- -------------- 7.9/12.6 MB 18.7 MB/s eta 0:00:01
   ---------------------------------------  12.3/12.6 MB 19.3 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 14.3 MB/s eta 0:00:00
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!python -m pip install azure_storage_blob

  Using cached azure_storage_blob-12.23.1-py3-none-any.whl.metadata (26 kB)
  Using cached azure_core-1.31.0-py3-none-any.whl.metadata (39 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached azure_storage_blob-12.23.1-py3-none-any.whl (405 kB)
Using cached azure_core-1.31.0-py3-none-any.whl (197 kB)
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ------------------------------------- -- 2.9/3.1 MB 18.7 MB/s eta 0:00:01
   ---------------------------------------- 3.1/3.1 MB 15.0 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
!python -m pip install python-dotenv

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [35]:
#Importing Necessary libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os

In [36]:
#Importing Necessary libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient
from dotenv import load_dotenv
import os


In [37]:
# Data Extraction
try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print('Data Extracted Successfully!')
except Exception as e:
    print(f"An erro occurred: {e}")

Data Extracted Successfully!


In [38]:
data.head(3)


,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [40]:
# Data cleaning and transformation
# Remove duplicates
data.drop_duplicates(inplace=True)

In [41]:
# Handle missing values( filling missing numeric values with the mean or median)
numeric_columns = data.select_dtypes(include =['float', 'int64']).columns
for col in numeric_columns:
    data.fillna({col: data[col].mean()}, inplace =True)

In [42]:
# Handling missing values (filling missing string object values with 'unknown')
String_columns = data.select_dtypes(include =['object']).columns
for col in String_columns:
    data.fillna({col: 'Unknown'}, inplace=True)

In [43]:
data['Date'] = pd.to_datetime(data['Date'])

In [44]:
#Create products table
products= data [['ProductName' ]].copy().drop_duplicates().reset_index(drop=True)
products.index.name ='ProductID'
products = products.reset_index()

In [45]:
customers =data [['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail']].copy().drop_duplicates().reset_index(drop=True)
customers.index.name ='CustomerID'
customers = customers.reset_index()#create customers table


In [46]:
# creating the staff
staff =data [['Staff_Name','Staff_Email']].copy().drop_duplicates().reset_index(drop=True)
staff.index.name ='StaffID'
staff = staff.reset_index()

In [47]:
# Creating the transactons tabel
transaction = data.merge(products, on=['ProductName'], how = 'left') \
                    .merge(customers, on = ['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail'], how ='left' ) \
                    .merge(staff, on= ['Staff_Name','Staff_Email'], how ='left' )
transaction.index.name ='TransactionID'
transaction = transaction.reset_index()\
                            [['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation','ProductID', \
                              'PaymentType', 'PromotionApplied', 'Weather', 'Temperature','StaffPerformanceRating','CustomerID', \
                                'CustomerFeedback', 'DeliveryTime_min','OrderType','StaffID','DayOfWeek','TotalSales']]

In [48]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   ProductID               1000 non-null   int64         
 6   PaymentType             1000 non-null   object        
 7   PromotionApplied        1000 non-null   bool          
 8   Weather                 1000 non-null   object        
 9   Temperature             1000 non-null   float64       
 10  StaffPerformanceRating  1000 non-null   object        
 11  CustomerID              1000 non-null   int64         
 12  CustomerFeedback        1000 non-null   object   

In [49]:
# Save data to as csv files
data.to_csv('clean_data.csv', index=False)
products.to_csv('products.csv', index=False)
customers.to_csv('customers.csv', index=False)
staff.to_csv('staff.csv', index=False)
transaction.to_csv('transaction.csv', index=False)

In [50]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [ ]:
# Clear cached variables
os.environ.pop('CONNECT_STR', None)
os.environ.pop('CONTAINER_NAME', None)

# Reload the .env file
load_dotenv(override=True)
connect_str = os.getenv('CONNECT_STR')
container_name = os.getenv('CONTAINER_NAME')

#print(connect_str)  # Ensure changes are reflected
#print(container_name)

In [ ]:
#print(f'connect_str: {connect_str}')

In [ ]:
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
from dotenv import load_dotenv
import os

# Load the environment variables
load_dotenv()

# Get the connection string and container name
connect_str = os.getenv('CONNECT_STR')
container_name = os.getenv('CONTAINER_NAME')

if not connect_str:
    raise ValueError("Azure Blob Storage connection string not found in environment variables.")

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Check and create the container if it doesn't exist
try:
    container_client = blob_service_client.create_container(container_name)
    print(f"Container '{container_name}' created.")
except ResourceExistsError:
    print(f"Container '{container_name}' already exists.")
    container_client = blob_service_client.get_container_client(container_name)

# Load data to Azure Blob Storage
files = [
    (data, 'rawdata/cleaned_zipco_transaction_data.csv'),
    (products, 'cleaneddata/product.csv'),
    (customers, 'cleaneddata/customers.csv'),
    (staff, 'cleaneddata/staff.csv'),
    (transaction, 'cleaneddata/transaction.csv')
]

# Upload the files to Azure Blob Storage
for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into Azure Blob Storage')
